# Setup

Make sure this notebook works well in both python 2 and 3, import a few common modules, ensure MatplotLib plots figures inline

In [789]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
%config InlineBackend.figure_format = 'retina' # for retina screens
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

Configure notebook to display all results in cell

In [790]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Revert to the last line of output only
# InteractiveShell.ast_node_interactivity = "last_expr"

# Load Data

In [791]:
import pandas as pd

houses_train = pd.read_csv('../Data/encoded_houses_train.csv')
houses_test = pd.read_csv('../Data/encoded_houses_test.csv')

In [792]:
print("houses_train dimensions: {}".format(houses_train.shape))
print("houses_test dimensions: {}".format(houses_test.shape))

houses_train dimensions: (1460, 224)
houses_test dimensions: (1459, 224)


In [793]:
pd.set_option("display.max_columns", 400)
houses_train.head(3)

,MSSubClass30,MSSubClass50,MSSubClass60,MSSubClass70,MSSubClass80,MSSubClass85,MSSubClass90,MSSubClass120,MSSubClass160,MSSubClass190,MSZoningFV,MSZoningRH,MSZoningRL,MSZoningRM,LotFrontage,LotArea,StreetPave,AlleyPave,AlleyNA,LotShapeIR1,LotShapeIR2,LotShapeIR3,LandContourBnk,LandContourHLS,LandContourLow,LotConfigCorner,LotConfigCulDSac,LotConfigFR2,LandSlopeMod,LandSlopeSev,NeighborhoodBlueste,NeighborhoodBrDale,NeighborhoodBrkSide,NeighborhoodClearCr,NeighborhoodCollgCr,NeighborhoodCrawfor,NeighborhoodEdwards,NeighborhoodGilbert,NeighborhoodIDOTRR,NeighborhoodMeadowV,NeighborhoodMitchel,NeighborhoodNAmes,NeighborhoodNoRidge,NeighborhoodNPkVill,NeighborhoodNridgHt,NeighborhoodNWAmes,NeighborhoodOldTown,NeighborhoodSWISU,NeighborhoodSawyer,NeighborhoodSawyerW,NeighborhoodSomerst,NeighborhoodStoneBr,NeighborhoodTimber,NeighborhoodVeenker,Condition1Feedr,Condition1Norm,Condition1RRAn,Condition1PosN,Condition1RRAe,BldgType2fmCon,BldgTypeDuplex,BldgTypeTwnhsE,BldgTypeTwnhs,HouseStyle1.5Fin,HouseStyle2Story,HouseStyle2.5Fin,HouseStyleSFoyer,HouseStyleSLvl,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyleGable,RoofStyleOther,RoofStyleHip,RoofMatlCompShg,RoofMatlTar&Grv,RoofMatlWood,Exterior1stBrick,Exterior1stCement,Exterior1stHdBoard,Exterior1stStucco,Exterior1stMetalSd,Exterior1stVinylSd,Exterior1stPlywood,Exterior1stWd Sdng,Exterior2ndBrick,Exterior2ndCement,Exterior2ndHdBoard,Exterior2ndStucco,Exterior2ndMetalSd,Exterior2ndVinylSd,Exterior2ndPlywood,Exterior2ndWd Sdng,Exterior2ndWd Shng,MasVnrTypeBrkFace,MasVnrTypeNone,MasVnrTypeStone,MasVnrArea,ExterQual4,ExterQual3,ExterQual2,ExterCondGd,ExterCondTA,ExterCondFa,ExterCondPo,FoundationCBlock,FoundationPConc,FoundationSlab,FoundationOther,BsmtQualGd,BsmtQualTA,BsmtQualFa,BsmtQualNA,BsmtCondTA,BsmtCondFa,BsmtCondNA,BsmtExposureAv,BsmtExposureMn,BsmtExposureNo,BsmtExposureNA,BsmtFinType1ALQ,BsmtFinType1BLQ,BsmtFinType1Rec,BsmtFinType1LwQ,BsmtFinType1Unf,BsmtFinType1NA,BsmtFinSF1,BsmtFinType2ALQ,BsmtFinType2BLQ,BsmtFinType2Rec,BsmtFinType2LwQ,BsmtFinType2Unf,BsmtFinType2NA,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,HeatingGasA,HeatingGasW,HeatingQC4,HeatingQC3,HeatingQC2,HeatingQC1,CentralAirY,ElectricalFuseA,ElectricalFuseFP,X1stFlrSF,X2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual4,KitchenQual3,KitchenQual2,TotRmsAbvGrd,FunctionalMin1,FunctionalMin2,FunctionalMod,FunctionalMaj,Fireplaces,FireplaceQuGd,FireplaceQuTA,FireplaceQuFa,FireplaceQuPo,FireplaceQuNA,GarageTypeAttchd,GarageTypeBasment,GarageTypeBuiltIn,GarageTypeDetchd,GarageTypeNA,GarageYrBlt,GarageFinishRFn,GarageFinishUnf,GarageFinishNA,GarageCars,GarageArea,GarageQualTA,GarageQualFaPo,GarageQualNA,GarageCondTA,GarageCondFaPo,GarageCondNA,PavedDriveP,PavedDriveN,WoodDeckSF,OpenPorchSF,EnclosedPorch,X3SsnPorch,ScreenPorch,PoolArea,PoolQCNA,FenceMnPrv,FenceGdWo,FenceMnWw,FenceNA,MiscFeatureShed,MiscVal,SaleTypeNew,SaleTypeCOD,SaleTypeCon,SaleConditionAbnorml,SaleConditionAlloca,SaleConditionFamily,SaleConditionPartial,Garage.interaction2.FaPo,Garage.interaction2.ExGd,Garage.interaction1.TA,Garage.interaction2.TA,Garage.interaction3.TA,Garage.interaction4.TA,Garage.interaction1.FaPo,new.old,Room.size,full.YrSold,QuarterSold,TotalBath,BathToBed,AvgHouseLivArea.ratio,SalePrice
0,-0.222645,-0.34576,1.905894,-0.206949,-0.203325,-0.117811,-0.192111,-0.251638,-0.229337,-0.144792,-0.215785,-0.105227,0.517956,-0.418812,-0.240150,-0.207071,0.064216,-0.169923,0.257733,-0.703962,-0.169923,-0.083017,-0.212287,-0.188246,-0.158945,-0.468578,-0.262234,-0.190187,-0.215785,-0.094752,-0.037024,-0.105227,-0.203325,-0.139784,2.954209,-0.190187,-0.27107,-0.239094,-0.161194,-0.108503,-0.186288,-0.426687,-0.169923,-0.07873,-0.235877,-0.229337,-0.289539,-0.131946,-0.230986,-0.205144,-0.250096,-0.131946,-0.163415,-0.087099,-0.242277,0.398273,-0.147237,-0.137218,-0.094752,-0.147237,-0.192111,-0.290925,-0.174141,-0.360475,1.509747,-0.114788,-0.161194,-0.215785,-0.651256,0.517023,1.05063

In [794]:
houses_train.info()
houses_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Columns: 224 entries, MSSubClass30 to SalePrice
dtypes: float64(224)
memory usage: 2.5 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Columns: 224 entries, MSSubClass30 to SalePrice
dtypes: float64(223), int64(1)
memory usage: 2.5 MB


Delete first column

In [795]:
# houses_train.drop('Unnamed: 0', axis=1, inplace=True, errors='raise')
# houses_test.drop('Unnamed: 0', axis=1, inplace=True, errors='raise')

# Run xgboost

### Create private training & test set

In [796]:
from sklearn.model_selection import train_test_split

seed = 10
test_ratio = 0.2

X = houses_train.loc[:, houses_train.columns != "SalePrice"].values # convert to np.array
y = houses_train.loc[:, houses_train.columns == "SalePrice"].values

# Take log of SalePrice
y = np.log(y + 1).ravel() # convert to 1D array for model fit (xxx, )


In [797]:
X_pr_train, X_pr_test, y_pr_train, y_pr_test = train_test_split(X, y, test_size=test_ratio, random_state=seed)

In [798]:
print(len(X_pr_train), "train +", len(X_pr_test), "test")

1168 train + 292 test


### Fit Model

In [799]:
from xgboost import XGBRegressor

xgb_clf = XGBRegressor(max_depth=3, 
                        learning_rate=0.1, 
                        n_estimators=1000, # Number of boosted trees to fit
                        silent=False, # print messages while running 
                        objective='reg:linear', 
                        booster='gbtree', # Specify which booster to use: gbtree, gblinear or dart
                        #for dart see http://xgboost.readthedocs.io/en/latest/tutorials/dart.html 
                        n_jobs=-1, # Number of parallel threads used to run xgboost. (replaces nthread)
                        gamma=0,  # Minimum loss reduction required to make a further partition on a leaf node of the tree.
                        min_child_weight=1, # Minimum sum of instance weight(hessian) needed in a child
                        max_delta_step=0, # Maximum delta step we allow each tree’s weight estimation to be
                        subsample=1, # Subsample ratio of the training instance
                        colsample_bytree=1, # Subsample ratio of columns when constructing each tree
                        colsample_bylevel=1, # Subsample ratio of columns for each split, in each level
                        reg_alpha=0, # L1 regularization term on weights
                        reg_lambda=1, # L2 regularization term on weights
                        scale_pos_weight=1, # Balancing of positive and negative weights
                        base_score=0.5, # The initial prediction score of all instances, global bias
                        random_state=743, 
                        missing=None) # Value in the data which needs to be present as a missing value. If None, defaults to np.nan



In [800]:
xgb_clf.fit(X_pr_train, y_pr_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=1000,
       n_jobs=-1, nthread=None, objective='reg:linear', random_state=743,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=False, subsample=1)

In [801]:
# make predictions for test data

y_pr_pred = xgb_clf.predict(X_pr_test)

Evaluate predictions

In [802]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_pr_test, y_pr_pred)
rmse = np.sqrt(mse)
rmse


0.12604123254806826

In [803]:
y_pr_test[1:5]
y_pr_pred[1:5]

array([ 12.1428719 ,  11.8277435 ,  12.01067193,  12.64109979])

array([ 12.22302151,  11.82423019,  11.93532276,  12.66820812], dtype=float32)

Save model to file

In [804]:
from sklearn.externals import joblib # More memory efficient than pickle for large numpy arrays

joblib.dump(xgb_clf, './Models/xgboost_model.pkl') 

['./Models/xgboost_model.pkl']

To load a model:

In [805]:
# xgb_clf_loaded = joblib.load('./Models/xgboost_model.pkl') 

### Model Tuning

In [806]:
from sklearn.model_selection import GridSearchCV

xgb_params = {'max_depth': 3,
              'learning_rate': 0.1, 
              'n_estimators': 100, 
              'objective': 'reg:linear'}

param_grid = {'max_depth': [3], 
              'learning_rate': [0.05], 
              'n_estimators': [2000], # Number of boosted trees to fit
              'objective': ['reg:linear'], 
              'booster': ['gbtree'], # Specify which booster to use: gbtree, gblinear or dart
              'n_jobs': [-1], # Number of parallel threads used to run xgboost. (replaces nthread)
              'gamma': [0],  # Minimum loss reduction required to make a further partition on a leaf node of the tree.
              'min_child_weight': [1], # Minimum sum of instance weight(hessian) needed in a child
              'max_delta_step': [0], # Maximum delta step we allow each tree’s weight estimation to be
              'subsample': [1], # Subsample ratio of the training instance
              'colsample_bytree': [0.5], # Subsample ratio of columns when constructing each tree
              'colsample_bylevel': [0.4], # Subsample ratio of columns for each split, in each level
              'reg_alpha': [0], # L1 regularization term on weights
              'reg_lambda': [1], # L2 regularization term on weights
              'scale_pos_weight': [1], # Balancing of positive and negative weights
              'base_score': [0.5], # The initial prediction score of all instances, global bias
              'silent': [True],
              'random_state': [10]}

optimized_xgb_clf = GridSearchCV(XGBRegressor(**xgb_params), # scikit-learn estimator interface 
                                 param_grid = param_grid, # Dictionary with parameters names (string) as keys
                                 scoring="neg_mean_squared_error", # controls what metric they apply to the estimators evaluated
                                 n_jobs=-1, # If True, the data is assumed to be identically distributed across the folds,
                                 iid=True, 
                                 refit=True, # Refit an estimator using the best found parameters (best_estimator_)
                                 cv=3, #integer, to specify the number of folds in a (Stratified)KFold. None -> default 3-fold cross validation
                                 verbose=10, # the higher, the more messages
                                 pre_dispatch="2*n_jobs", # number of jobs that get dispatched during parallel execution
                                 error_score="raise", 
                                 return_train_score=False) #If False, the cv_results_ attribute will not include training scores

In [807]:
# param_grid = {'max_depth': [3, 4, 5], 
#               'learning_rate': [0.07], 
#               'n_estimators': [1500, 1800, 2000, 2200, 2400], # Number of boosted trees to fit
#               'objective': ['reg:linear'], 
#               'booster': ['gbtree'], # Specify which booster to use: gbtree, gblinear or dart
#               'n_jobs': [-1], # Number of parallel threads used to run xgboost. (replaces nthread)
#               'gamma': [0, 0.5],  # Minimum loss reduction required to make a further partition on a leaf node of the tree.
#               'min_child_weight': [1], # Minimum sum of instance weight(hessian) needed in a child
#               'max_delta_step': [0], # Maximum delta step we allow each tree’s weight estimation to be
#               'subsample': [1], # Subsample ratio of the training instance
#               'colsample_bytree': [0.6], # Subsample ratio of columns when constructing each tree
#               'colsample_bylevel': [0.3], # Subsample ratio of columns for each split, in each level
#               'reg_alpha': [0], # L1 regularization term on weights
#               'reg_lambda': [1], # L2 regularization term on weights
#               'scale_pos_weight': [1], # Balancing of positive and negative weights
#               'base_score': [0.5], # The initial prediction score of all instances, global bias
#               'silent': [True],
#               'random_state': [10]}

Inspect the grid

In [808]:
optimized_xgb_clf

GridSearchCV(cv=3, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [3], 'learning_rate': [0.05], 'n_estimators': [2000], 'objective': ['reg:linear'], 'booster': ['gbtree'], 'n_jobs': [-1], 'gamma': [0], 'min_child_weight': [1], 'max_delta_step': [0], 'subsample': [1], 'colsample_bytree': [0.5], 'colsample_bylevel': [0.4], 'reg_alpha': [0], 'reg_lambda': [1], 'scale_pos_weight': [1], 'base_score': [0.5], 'silent': [True], 'random_state': [10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring='neg_mean_squared

Run grid tuning

In [809]:
optimized_xgb_clf.fit(X_pr_train, y_pr_train)

Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] base_score=0.5, booster=gbtree, colsample_bylevel=0.4, colsample_bytree=0.5, gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=2000, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1 
[CV] base_score=0.5, booster=gbtree, colsample_bylevel=0.4, colsample_bytree=0.5, gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=2000, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1 
[CV] base_score=0.5, booster=gbtree, colsample_bylevel=0.4, colsample_bytree=0.5, gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=3, min_child_weight=1, n_estimators=2000, n_jobs=-1, objective=reg:linear, random_state=10, reg_alpha=0, reg_lambda=1, scale_pos_weight=1, silent=True, subsample=1 
[CV]  base_score=0.5, booster

[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    6.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    6.0s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'max_depth': [3], 'learning_rate': [0.05], 'n_estimators': [2000], 'objective': ['reg:linear'], 'booster': ['gbtree'], 'n_jobs': [-1], 'gamma': [0], 'min_child_weight': [1], 'max_delta_step': [0], 'subsample': [1], 'colsample_bytree': [0.5], 'colsample_bylevel': [0.4], 'reg_alpha': [0], 'reg_lambda': [1], 'scale_pos_weight': [1], 'base_score': [0.5], 'silent': [True], 'random_state': [10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
       scoring='neg_mean_squared

The best combination of parameters is:

In [810]:
optimized_xgb_clf.best_params_

{'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 0.4,
 'colsample_bytree': 0.5,
 'gamma': 0,
 'learning_rate': 0.05,
 'max_delta_step': 0,
 'max_depth': 3,
 'min_child_weight': 1,
 'n_estimators': 2000,
 'n_jobs': -1,
 'objective': 'reg:linear',
 'random_state': 10,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'silent': True,
 'subsample': 1}

In [811]:
## The best score is
optimized_xgb_clf.best_score_ # that's the training score so not meaningful

-0.015844460670569646

In [812]:
# optimized_xgb_clf.cv_results_

In [813]:
# make predictions for test data

y_pr_pred = optimized_xgb_clf.predict(X_pr_test)

Evaluate predictions

In [814]:
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_pr_test, y_pr_pred)
rmse = np.sqrt(mse)
rmse


0.1247490807943749

In [815]:
y_pr_test[1:5]
y_pr_pred[1:5]

array([ 12.1428719 ,  11.8277435 ,  12.01067193,  12.64109979])

array([ 12.22820187,  11.83184052,  11.93189049,  12.66695595], dtype=float32)